## Mathematically Satisfying Wheel of the Year

My spouse and I invented this holiday scheme the year that they started grad school*, 2011, when we were living in Providence, RI. We're both atheists, but we love celebrations, and we think traditions are important. Also, I had spent a hot minute in my teens as a Wiccan, and I really loved the "Wheel of the Year" style of holidays, where you celebrate the solar solstices and equinoxes as major holidays, and dates directly in between those holidays as minor holidays. This "Quarters and Cross-Quarters" approach means you have a holiday about every other month! 

They really liked that to, but they added a fun extra bit -- what if the major holidays were "maximum festivity", like 100% festive, and the minor holidays were half-power festivity, 50% festive? We quickly realized that we could probably map a trigonometric function to that, and that that would also mean that directly in between each holiday you would have a festivity nadir -- a time of absolute minimum festivity. 

Neither of us are good enough mathematicians to figure out how to derive that exact function, so we asked a math professor friend. The function he responded with was: 

$$
f(x) = \left[ \frac{1}{8}(2 - \sqrt{2}) + \frac{1}{4}(2 + \sqrt{2}) \cos(2x) + \frac{1}{8}(2 - \sqrt{2}) \cos(4x) \right]^2
$$


An intimidatingly impressive function, to be sure. You can see (and check!) his work in the PDF in this workspace. 

### The need for code

Obviously, we now need to be able to calculate the exact amount of festivity at any given moment, so now we have to write some python code. To be completely blunt, we haven't ever actually finished this project. In fact, finishing the project itself has become sort of a yearly tradition, because in the run up to every winter solstice, we have a little freakout about the fact that we STILL haven't finished this software, and we can't therefore direct people to our envisioned website where they can put in a date and get back the festivity index. 

Some of the reason we haven't finished is because it's actually kind of hard, and some of it is because of typical scope creep. The thing that's actually HARD about this is that we wanted to pin it to the "real" solar holidays: Winter Solstice is when the Solar Declination is -23.5 degrees. Because these are real objects in three dimensional space, and gravity makes things wobbly, and the calendar isn't really "right" anyway, Winter Solstice as an astronomical event isn't actually at the same time every year. It's at or around December 21. In the Northern Hemisphere, that is. 
The scope creep was about wanting to add additional bells and whistles. We actually started out doing this in 2012 in a jupyter notebook using the package pyEphem to calculate the exact major solar holidays based on solar declination, and then I learned R, and we wanted to be able to plot the festivity function using ggplot2, but getting the holidays was much much harder in R. 

I hate to say it, but LLMs have actually ended up helping us get much closer to finishing this project. That, and limiting our scope and making some concessions. Instead of calculating the exact solar holidays each year, we're just gonna call the dates for each major holiday, and then use those to get the dates for the other holidays. It won't be perfect, but it's about right. Once I decided to do that, it was relatively easy to get ChatGPT 4o to write the functions, with only some minimal editing from me. 

## Convert Major holiday dates into Radians

In [14]:
from datetime import datetime, timedelta
import math

def date_as_x(date: datetime) -> float:
    """
    Converts a date within the year to radians, mapping the time interval
    linearly from 0 to pi for the corresponding quarter.
    
    Parameters:
    - date: The date to be converted (datetime object).
    
    Returns:
    - A float representing the date in radians, ranging from 0 to pi.
    """
    year = date.year
    
    # Define the boundaries for each quarter
    winter_solstice = datetime(year - 1, 12, 21)  # Winter Solstice of the previous year
    spring_equinox = datetime(year, 3, 20)        # Spring Equinox
    summer_solstice = datetime(year, 6, 21)       # Summer Solstice
    fall_equinox = datetime(year, 9, 22)          # Fall Equinox
    next_winter_solstice = datetime(year, 12, 21) # Winter Solstice of the current year

    # Determine the quarter based on the date
    if winter_solstice <= date <= spring_equinox:
        start_date, end_date = winter_solstice, spring_equinox
    elif spring_equinox <= date <= summer_solstice:
        start_date, end_date = spring_equinox, summer_solstice
    elif summer_solstice <= date <= fall_equinox:
        start_date, end_date = summer_solstice, fall_equinox
    elif fall_equinox <= date <= next_winter_solstice:
        start_date, end_date = fall_equinox, next_winter_solstice
    else:
        raise ValueError(f"Date must be within the current year's range.")

    # Calculate total days in the current quarter
    total_days = (end_date - start_date).days

    # Calculate the number of days since the start of the quarter
    days_since_start = (date - start_date).days

    # Map the days to radians, linearly scaling from 0 to pi
    radians = (days_since_start / total_days) * math.pi
    
    return radians


#### Test

In [2]:
date_as_x(datetime(2024, 8, 14))

test1 = date_as_x(datetime(2024, 8, 14))
test2 = date_as_x(datetime(2021, 12, 21))
test3 = date_as_x(datetime.now())

## Define cosine function to get the Festivity of the date

In [3]:
def cosine_formula(x: float) -> float:
    """
    Applies the given cosine-based formula to the input radians value.
    
    f(x) = (1/8 * (2 - sqrt(2))) + (1/4 * (2 + (sqrt(2) * cos(2x)))) + (1/8 * (2 - sqrt(2) * cos(4x)))
    
    Parameters:
    - x: The input radians value (float).
    
    Returns:
    - The result of the formula (float).
    """
    term1 = 1/8 * (2 - math.sqrt(2))
    term2 = 1/4 * (2 + (math.sqrt(2) * math.cos(2 * x)))
    term3 = 1/8 * (2 - math.sqrt(2) * math.cos(4 * x))
    
    result = term1 + term2 + term3
    return result


#### Test

In [4]:
print(cosine_formula(test1))
print(cosine_formula(test2))
print(cosine_formula(test3))

0.42058658433523405
1.0
0.6483202558683268


## Get the series of dates for a year

These have known festivity values, and we just need to calculate when exactly they are. 

In [15]:
def get_midpoint_date(date1: datetime, date2: datetime) -> datetime:
    """
    Returns the midpoint date between two datetime objects.
    """
    return date1 + (date2 - date1) / 2

def get_holidays(year: int) -> list:
    """
    Generates a series of dates for the specified year, including
    major holidays, cross-quarters, and nadirs.
    
    Parameters:
    - year: The year for which to generate the holidays (int).
    
    Returns:
    - A list of 16 dates representing the holidays.
    """
    # Major holidays
    prev_winter_solstice = datetime(year - 1, 12, 21)
    spring_equinox = datetime(year, 3, 20)
    summer_solstice = datetime(year, 6, 21)
    fall_equinox = datetime(year, 9, 22)
    winter_solstice = datetime(year, 12, 21)

    major_holidays = [
        prev_winter_solstice,
        fall_equinox,
        winter_solstice,
        spring_equinox,
        summer_solstice
    ]
    major_holidays = sorted(major_holidays)
    # Calculate cross-quarters
    cross_quarters = [
        get_midpoint_date(major_holidays[i], major_holidays[i + 1])
        for i in range(len(major_holidays) - 1)
    ]
    cross_quarters.append(get_midpoint_date(winter_solstice, prev_winter_solstice))
    cross_quarters = sorted(cross_quarters)
    
    # Complete series of major and minor holidays
    holidays = []
    for i in range(len(major_holidays)):
        holidays.append(major_holidays[i])
        holidays.append(cross_quarters[i])

    holidays = sorted(holidays)
    # Calculate nadirs
    nadirs = [
        get_midpoint_date(holidays[i], holidays[i + 1])
        for i in range(len(holidays) - 1)
    ]

    # Combine major holidays, cross-quarters, and nadirs into the final series
    final_series = []
    for i in range(len(holidays) - 1):
        final_series.append(holidays[i])
        final_series.append(nadirs[i])
    final_series.append(holidays[-1])  # Add the last major holiday
    final_series = sorted(final_series)

    final_series.pop(0)
    final_series = list(set(final_series))
    
    # Convert to readable format
    formatted_series = [date.strftime("%Y/%m/%d") for date in final_series]

    return formatted_series


Test

In [16]:
# Example usage: Get holidays for 2024
holidays_2024 = get_holidays(2024)
print(holidays_2024)

['2024/06/21', '2024/08/06', '2024/02/04', '2024/02/26', '2024/05/28', '2024/11/06', '2024/05/05', '2024/10/14', '2024/01/12', '2024/11/28', '2024/12/21', '2024/07/14', '2024/04/12', '2024/08/29', '2024/09/22', '2024/03/20']


In [9]:
len(holidays_2024)

16

## Create a table of holidays aong with celebrations

I've got a list of the holidays, and the traditions we established for each of them. I want to add the dates for the calendar year so that we know what day to celebrate what!

In [17]:
import pandas as pd

hol = pd.read_csv("holidays.csv")

def add_dates_to_holiday_table(year: int, holiday_table: pd.DataFrame) -> pd.DataFrame:
    """
    Adds a column of dates to the given holiday table based on the specified year.
    
    Parameters:
    - year: The year for which to generate the holiday dates (int).
    - holiday_table: The DataFrame containing holiday names and celebrations.
    
    Returns:
    - A DataFrame with an additional column for the calculated holiday dates.
    """
    # Get the list of 16 holiday dates for the specified year
    holiday_dates = get_holidays(year)
    
    # Add the dates to the DataFrame as a new column
    holiday_table['Date'] = holiday_dates
    
    return holiday_table


Test

In [18]:
## example: 
add_dates_to_holiday_table(2024, hol)

,Index,Holiday Name,Celebrations,Date
0,0,First Winter Nadir,"getting rid of our Christmas tree, preparing o...",2024/06/21
1,1,Winter Cross Quarter,getting tight on rum and watching The Rocky Ho...,2024/08/06
2,2,Second Winter Nadir,making dental appointments.,2024/02/04
3,3,Vernal Equinox,eating eggs benedict and jousting peeps in the...,2024/02/26
4,4,First Spring Nadir,giving away clothes that don't fit or that we ...,2024/05/28
5,5,Spring Cross Quarter,singing Solidarity Forever and going out for a...,2024/11/06
6,6,Second Spring Nadir,getting rid of things we wouldn't want to move...,2024/05/05
7,7,Summer Solstice,putting up paper lanterns and watching a produ...,2024/10/14
8,8,First Summer Nadir,evaluating family finances.,2024/01/12
9,9,Summer Cross Quarter,making fancy cheese boards with bread and wine...,2024/11/28
